# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]
    START WITH 1
    INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [REGIOS] [VARCHAR](100),
    [CODE_1] [VARCHAR](100),
	[CODE_42] [VARCHAR](100),
	[NAAM_43] [VARCHAR](100),
	[INWONERTAL_50] [VARCHAR](100)
);
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]') AND TYPE = 'SO')
CREATE SEQUENCE [SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]
    START WITH 1
    INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]),
    [VRCODE] VARCHAR(5),
    [GMCODE] VARCHAR(10),
	[INHABITANTS] BIGINT,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE NAME='IX_VWS_VWSSTATIC_CBS_INHABITANTS_PER_MUNICIPALITY')
CREATE NONCLUSTERED INDEX [IX_VWS_VWSSTATIC_CBS_INHABITANTS_PER_MUNICIPALITY] ON [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [DATE_LAST_INSERTED], 
    [VRCODE], 
    [GMCODE]
) INCLUDE ([INHABITANTS]);
GO

# **STORED PROCEDURES**

---

De SP die de data van VWSStage naar VWSStatic kopieert wordt uitgebreid met een join naar de tabel Recode\_Municipality om de gegevens direct om te zetten naar de nieuwe gemeentelijke indeling.

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [dbo].[SP_INHABITANTS_PER_REGION_STATIC]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]  (
        [VRCODE], 
        [GMCODE], 
        [INHABITANTS]
    )
    SELECT
        ISNULL(B.[VR_CODE], [CODE_42]),
        ISNULL(B.[GM_CODE_NEW], [CODE_1]),
        CAST(ROUND(SUM(CAST(A.[INWONERTAL_50] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT)
    FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] A
        LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE] = 1 
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[CODE_1] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY])
    GROUP BY ISNULL(B.[VR_CODE],[CODE_42]), ISNULL(B.[GM_CODE_NEW],[CODE_1])
END;
GO